In [1]:
import numpy as np

import sys
sys.path.insert(
    1,
    '/home/trduong/Data/interpretable_machine_learning/Source Code/my_work/lib'
)

import numpy as np
import pandas as pd
import logging
import incremental_ps_score_estimator as ipse
import utils


from sklearn.ensemble import RandomForestClassifier, GradientBoostingRegressor, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from geneticalgorithm import geneticalgorithm as ga


In [2]:
file_path = "https://msalicedatapublic.blob.core.windows.net/datasets/Pricing/pricing_sample.csv"
train_data = pd.read_csv(file_path)

In [3]:
train_data['treatment'] = np.where(train_data['price'] == 1, 1, 0)
train_data['price'] = np.where(train_data['price'] == 1, 1, 0.85)

In [4]:
outcome = "demand"
treatment = "treatment"
col = list(train_data.columns)
col.remove("price")
print(col)

cov = col[:]
cov.remove(treatment)
cov.remove(outcome)
cov.remove('income')
print(cov)

features = col[:]
features.remove(outcome)

print(features)

['account_age', 'age', 'avg_hours', 'days_visited', 'friends_count', 'has_membership', 'is_US', 'songs_purchased', 'income', 'demand', 'treatment']
['account_age', 'age', 'avg_hours', 'days_visited', 'friends_count', 'has_membership', 'is_US', 'songs_purchased']
['account_age', 'age', 'avg_hours', 'days_visited', 'friends_count', 'has_membership', 'is_US', 'songs_purchased', 'income', 'treatment']


In [37]:
from sklearn.model_selection import train_test_split

train_data, test = train_test_split(train_data, test_size=0.3, random_state = 1)
train_data = train.reset_index(drop=True)
test = test.reset_index(drop=True)



In [38]:
## Fit treatment
model_t = LogisticRegression()
model_t.fit(train_data[cov], train_data[treatment])

train_data['p1'] = model_t.predict_proba(train_data[cov])[:,1]
train_data['p0'] = 1 - train_data['p1']

train_data["prediction"] = np.where(train_data["p1"] >= 0.5, 1, 0)

## Fit outcome
model_y = GradientBoostingRegressor(random_state=0, n_estimators = 1000)
model_y.fit(train_data[features], train_data[outcome])

    

GradientBoostingRegressor(n_estimators=1000, random_state=0)

In [39]:
train_data.shape

(5000, 15)

## Optimization

In [40]:
def gamma_fn(X):
    return -3 - 14 * (X["income"] < 1)

def beta_fn(X):
    return 20 + 0.5 * (X["avg_hours"]) + 5 * (X["days_visited"] > 4)

def demand_fn(data, T):
    Y = gamma_fn(data) * T + beta_fn(data)
    return Y

def true_te(x, n, stats):
    if x < 1:
        subdata = df[df["income"] < 1].sample(n=n, replace=True)
    else:
        subdata = df[df["income"] >= 1].sample(n=n, replace=True)
    te_array = subdata["price"] * gamma_fn(subdata) / (subdata["demand"])
    if stats == "mean":
        return np.mean(te_array)
    elif stats == "median":
        return np.median(te_array)
    elif isinstance(stats, int):
        return np.percentile(te_array, stats)
    
def revenue_fn(data, discount_level1, discount_level2, baseline_T, policy):
    policy_price = baseline_T * (1 - discount_level1) * policy + baseline_T * (1 - discount_level2) * (1 - policy)
    demand = demand_fn(data, policy_price)
    rev = demand * policy_price
    return rev

def fitness_function(delta, solution_idx):
    propensity_score = (delta * train_data['p1']) / (delta * train_data['p1'] + train_data['p0'])
    policy = np.where(propensity_score >= 0.5, 1, 0)
    return np.mean(revenue_fn(train_data,  0., 0.15, 1, policy))


In [41]:
train_data.head()

,account_age,age,avg_hours,days_visited,friends_count,has_membership,is_US,songs_purchased,income,price,demand,treatment,p1,p0,prediction
0,5,59,3.267122,3,13,0,1,7.483683,0.844633,0.85,8.033561,0,0.421064,0.578936,0
1,2,50,5.929330,6,13,0,1,2.839970,0.788542,1.00,10.964665,1,0.438462,0.561538,0
2,1,26,6.296786,5,9,0,1,3.244757,1.400923,1.00,25.148393,1,0.435456,0.564544,0
3,1,31,8.989537,3,11,0,1,1.703305,1.090044,0.85,21.794768,0,0.429353,0.570647,0
4,1,42,7.448348,7,4,1,1,7.506403,0.669766,1.00,11.724174,1,0.412282,0.587718,0


In [42]:
#0: discount, level 2, 1: no discount - level 1
np.mean(revenue_fn(train_data,  0.0, 0.15, 1, train_data.treatment.values))

14.243903028905738

In [43]:
np.mean(train_data['demand']*train_data['price'])

14.314937528905737

## PyGrad

In [44]:
import pygad
import numpy

sols = []

function_inputs = np.random.rand(train_data.shape[0]) # Function inputs.

num_generations = 5000 # Number of generations.
num_parents_mating = 500 # Number of solutions to be selected as parents in the mating pool.

# To prepare the initial population, there are 2 ways:
# 1) Prepare it yourself and pass it to the initial_population parameter. This way is useful when the user wants to start the genetic algorithm with a custom initial population.
# 2) Assign valid integer values to the sol_per_pop and num_genes parameters. If the initial_population parameter exists, then the sol_per_pop and num_genes parameters are useless.
sol_per_pop = 1500 # Number of solutions in the population.
num_genes = len(function_inputs)

init_range_low = -1
init_range_high = 1

parent_selection_type = "sss" # Type of parent selection.
keep_parents = -1 # Number of parents to keep in the next population. -1 means keep all parents and 0 means keep nothing.

crossover_type = "two_points" # Type of the crossover operator.

# Parameters of the mutation operation.
mutation_type = "random" # Type of the mutation operator.
mutation_percent_genes = 30 # Percentage of genes to mutate. This parameter has no action if the parameter mutation_num_genes exists or when mutation_type is None.

last_fitness = 0
def callback_generation(ga_instance):
    global last_fitness
    global sols
    print("Generation = {generation}".format(generation=ga_instance.generations_completed))
    print("Fitness    = {fitness}".format(fitness=ga_instance.best_solution()[1]))
    print("Change     = {change}".format(change=ga_instance.best_solution()[1] - last_fitness))
    print("\n")
    last_fitness = ga_instance.best_solution()[1]
    solution, solution_fitness, solution_idx = ga_instance.best_solution()
    sols.append(solution)

print("Running model")
# Creating an instance of the GA class inside the ga module. Some parameters are initialized within the constructor.
ga_instance = pygad.GA(num_generations=num_generations,
                       num_parents_mating=num_parents_mating, 
                       fitness_func=fitness_function,
                       sol_per_pop=sol_per_pop, 
                       num_genes=num_genes,
                       init_range_low=init_range_low,
                       init_range_high=init_range_high,
                       parent_selection_type=parent_selection_type,
                       keep_parents=keep_parents,
                       crossover_type=crossover_type,
                       mutation_type=mutation_type,
                       mutation_percent_genes=mutation_percent_genes,
                       callback_generation=callback_generation)

# Running the GA to optimize the parameters of the function.
ga_instance.run()


Running model
Generation = 1
Fitness    = 13.45870315224051
Change     = 13.45870315224051


Generation = 2
Fitness    = 13.496530788625707
Change     = 0.03782763638519704


Generation = 3
Fitness    = 13.532816672948679
Change     = 0.03628588432297164


Generation = 4
Fitness    = 13.562254913028124
Change     = 0.029438240079445066


Generation = 5
Fitness    = 13.592520704065752
Change     = 0.030265791037628276


Generation = 6
Fitness    = 13.624943408237863
Change     = 0.03242270417211124


Generation = 7
Fitness    = 13.637525246315473
Change     = 0.01258183807760993


Generation = 8
Fitness    = 13.66305310312087
Change     = 0.02552785680539671


Generation = 9
Fitness    = 13.690262254511152
Change     = 0.02720915139028257


Generation = 10
Fitness    = 13.723494603212085
Change     = 0.033232348700932945


Generation = 11
Fitness    = 13.73033801579442
Change     = 0.006843412582334452


Generation = 12
Fitness    = 13.751126333343237
Change     = 0.020788317548817048



Generation = 104
Fitness    = 14.211734904252785
Change     = 0.0


Generation = 105
Fitness    = 14.212539575068025
Change     = 0.0008046708152402715


Generation = 106
Fitness    = 14.214398710752842
Change     = 0.0018591356848176588


Generation = 107
Fitness    = 14.214398710752842
Change     = 0.0


Generation = 108
Fitness    = 14.221048857566847
Change     = 0.006650146814004998


Generation = 109
Fitness    = 14.221048857566847
Change     = 0.0


Generation = 110
Fitness    = 14.221048857566847
Change     = 0.0


Generation = 111
Fitness    = 14.221048857566847
Change     = 0.0


Generation = 112
Fitness    = 14.22603458499455
Change     = 0.004985727427703424


Generation = 113
Fitness    = 14.22603458499455
Change     = 0.0


Generation = 114
Fitness    = 14.22603458499455
Change     = 0.0


Generation = 115
Fitness    = 14.22950286334164
Change     = 0.003468278347089182


Generation = 116
Fitness    = 14.22980401603806
Change     = 0.00030115269641939335


Generation = 11

Fitness    = 14.297701783983683
Change     = 0.0


Generation = 216
Fitness    = 14.297701783983683
Change     = 0.0


Generation = 217
Fitness    = 14.297701783983683
Change     = 0.0


Generation = 218
Fitness    = 14.298743474695092
Change     = 0.0010416907114088048


Generation = 219
Fitness    = 14.298743474695092
Change     = 0.0


Generation = 220
Fitness    = 14.298743474695092
Change     = 0.0


Generation = 221
Fitness    = 14.298743474695092
Change     = 0.0


Generation = 222
Fitness    = 14.298743474695092
Change     = 0.0


Generation = 223
Fitness    = 14.300730847673513
Change     = 0.00198737297842122


Generation = 224
Fitness    = 14.300730847673513
Change     = 0.0


Generation = 225
Fitness    = 14.300730847673513
Change     = 0.0


Generation = 226
Fitness    = 14.301092143681844
Change     = 0.0003612960083305694


Generation = 227
Fitness    = 14.301092143681844
Change     = 0.0


Generation = 228
Fitness    = 14.301092143681844
Change     = 0.0


Generation = 

Generation = 331
Fitness    = 14.339132343147323
Change     = 0.0


Generation = 332
Fitness    = 14.339132343147323
Change     = 0.0


Generation = 333
Fitness    = 14.339132343147323
Change     = 0.0


Generation = 334
Fitness    = 14.339132343147323
Change     = 0.0


Generation = 335
Fitness    = 14.339132343147323
Change     = 0.0


Generation = 336
Fitness    = 14.339132343147323
Change     = 0.0


Generation = 337
Fitness    = 14.339132343147323
Change     = 0.0


Generation = 338
Fitness    = 14.339132343147323
Change     = 0.0


Generation = 339
Fitness    = 14.339132343147323
Change     = 0.0


Generation = 340
Fitness    = 14.339132343147323
Change     = 0.0


Generation = 341
Fitness    = 14.339132343147323
Change     = 0.0


Generation = 342
Fitness    = 14.339132343147323
Change     = 0.0


Generation = 343
Fitness    = 14.339132343147323
Change     = 0.0


Generation = 344
Fitness    = 14.339132343147323
Change     = 0.0


Generation = 345
Fitness    = 14.339132343147323

Generation = 449
Fitness    = 14.356437197708798
Change     = 0.0


Generation = 450
Fitness    = 14.356437197708798
Change     = 0.0


Generation = 451
Fitness    = 14.356437197708798
Change     = 0.0


Generation = 452
Fitness    = 14.356437197708798
Change     = 0.0


Generation = 453
Fitness    = 14.356437197708798
Change     = 0.0


Generation = 454
Fitness    = 14.356437197708798
Change     = 0.0


Generation = 455
Fitness    = 14.356437197708798
Change     = 0.0


Generation = 456
Fitness    = 14.356437197708798
Change     = 0.0


Generation = 457
Fitness    = 14.356437197708798
Change     = 0.0


Generation = 458
Fitness    = 14.356437197708798
Change     = 0.0


Generation = 459
Fitness    = 14.356437197708798
Change     = 0.0


Generation = 460
Fitness    = 14.356437197708798
Change     = 0.0


Generation = 461
Fitness    = 14.356437197708798
Change     = 0.0


Generation = 462
Fitness    = 14.356437197708798
Change     = 0.0


Generation = 463
Fitness    = 14.356437197708798

...

Fitness    = 14.412573272889075
Change     = 0.0


Generation = 1760
Fitness    = 14.412573272889075
Change     = 0.0


Generation = 1761
Fitness    = 14.412573272889075
Change     = 0.0


Generation = 1762
Fitness    = 14.412573272889075
Change     = 0.0


Generation = 1763
Fitness    = 14.412573272889075
Change     = 0.0


Generation = 1764
Fitness    = 14.412573272889075
Change     = 0.0


Generation = 1765
Fitness    = 14.412573272889075
Change     = 0.0


Generation = 1766
Fitness    = 14.412573272889075
Change     = 0.0


Generation = 1767
Fitness    = 14.412573272889075
Change     = 0.0


Generation = 1768
Fitness    = 14.412573272889075
Change     = 0.0


Generation = 1769
Fitness    = 14.412573272889075
Change     = 0.0


Generation = 1770
Fitness    = 14.412573272889075
Change     = 0.0


Generation = 1771
Fitness    = 14.412573272889075
Change     = 0.0


Generation = 1772
Fitness    = 14.412573272889075
Change     = 0.0


Generation = 1773
Fitness    = 14.412573272889075
Ch

Generation = 1879
Fitness    = 14.41388254869435
Change     = 0.0


Generation = 1880
Fitness    = 14.41388254869435
Change     = 0.0


Generation = 1881
Fitness    = 14.41388254869435
Change     = 0.0


Generation = 1882
Fitness    = 14.41388254869435
Change     = 0.0


Generation = 1883
Fitness    = 14.41388254869435
Change     = 0.0


Generation = 1884
Fitness    = 14.41388254869435
Change     = 0.0


Generation = 1885
Fitness    = 14.41388254869435
Change     = 0.0


Generation = 1886
Fitness    = 14.41388254869435
Change     = 0.0


Generation = 1887
Fitness    = 14.41388254869435
Change     = 0.0


Generation = 1888
Fitness    = 14.41388254869435
Change     = 0.0


Generation = 1889
Fitness    = 14.41388254869435
Change     = 0.0


Generation = 1890
Fitness    = 14.41388254869435
Change     = 0.0


Generation = 1891
Fitness    = 14.41388254869435
Change     = 0.0


Generation = 1892
Fitness    = 14.41388254869435
Change     = 0.0


Generation = 1893
Fitness    = 14.41388254869435

Change     = 0.0


Generation = 2000
Fitness    = 14.41388254869435
Change     = 0.0


Generation = 2001
Fitness    = 14.41388254869435
Change     = 0.0


Generation = 2002
Fitness    = 14.41388254869435
Change     = 0.0


Generation = 2003
Fitness    = 14.41388254869435
Change     = 0.0


Generation = 2004
Fitness    = 14.41388254869435
Change     = 0.0


Generation = 2005
Fitness    = 14.41388254869435
Change     = 0.0


Generation = 2006
Fitness    = 14.41388254869435
Change     = 0.0


Generation = 2007
Fitness    = 14.41388254869435
Change     = 0.0


Generation = 2008
Fitness    = 14.41388254869435
Change     = 0.0


Generation = 2009
Fitness    = 14.41388254869435
Change     = 0.0


Generation = 2010
Fitness    = 14.41388254869435
Change     = 0.0


Generation = 2011
Fitness    = 14.41388254869435
Change     = 0.0


Generation = 2012
Fitness    = 14.41388254869435
Change     = 0.0


Generation = 2013
Fitness    = 14.41388254869435
Change     = 0.0


Generation = 2014
Fitness    

Generation = 2119
Fitness    = 14.418979004549637
Change     = 0.0


Generation = 2120
Fitness    = 14.418979004549637
Change     = 0.0


Generation = 2121
Fitness    = 14.418979004549637
Change     = 0.0


Generation = 2122
Fitness    = 14.418979004549637
Change     = 0.0


Generation = 2123
Fitness    = 14.418979004549637
Change     = 0.0


Generation = 2124
Fitness    = 14.418979004549637
Change     = 0.0


Generation = 2125
Fitness    = 14.418979004549637
Change     = 0.0


Generation = 2126
Fitness    = 14.418979004549637
Change     = 0.0


Generation = 2127
Fitness    = 14.418979004549637
Change     = 0.0


Generation = 2128
Fitness    = 14.418979004549637
Change     = 0.0


Generation = 2129
Fitness    = 14.418979004549637
Change     = 0.0


Generation = 2130
Fitness    = 14.418979004549637
Change     = 0.0


Generation = 2131
Fitness    = 14.418979004549637
Change     = 0.0


Generation = 2132
Fitness    = 14.418979004549637
Change     = 0.0


Generation = 2133
Fitness    = 14.

KeyboardInterrupt: 

In [45]:
sols

[array([ 1.90740576, -0.44852508, -1.12407538, ..., -6.0330004 ,
        -5.97021603,  5.37544962]),
 array([ 3.42597731, -0.37472711, -2.09675276, ..., -5.43300178,
        -5.39559384,  4.03580992]),
 array([ 3.42172135, -1.96784581, -2.51383   , ..., -7.28520605,
        -4.69052622,  4.67742892]),
 array([ 3.36843891, -2.40861221, -1.7685175 , ..., -6.77684403,
        -4.77233841,  4.69177428]),
 array([ 2.65305164, -0.26133511, -2.36288288, ..., -5.31214186,
        -3.57850109,  4.65610528]),
 array([ 3.42172135, -1.96784581, -2.51383   , ..., -7.28520605,
        -4.69052622,  4.67742892]),
 array([ 2.65305164, -0.44852508, -1.59305797, ..., -7.56599715,
        -5.05996306,  4.0032287 ]),
 array([ 3.36843891, -2.55645897, -1.94976284, ..., -5.88009156,
        -3.68147817,  6.33235099]),
 array([ 3.53678951, -2.55645897, -1.2545177 , ..., -3.58766132,
        -4.41981194,  6.17897917]),
 array([ 1.90740576, -0.44852508, -1.09758616, ..., -7.53070468,
        -5.99259235,  5.37

In [46]:
solution, solution_fitness, solution_idx = ga_instance.best_solution()
print("Parameters of the best solution : {solution}".format(solution=solution))
print("Fitness value of the best solution = {solution_fitness}".format(solution_fitness=solution_fitness))

Parameters of the best solution : [ 1.90740576 -0.44852508 -2.07174024 ... -6.56301649 -5.06847645
  5.37544962]
Fitness value of the best solution = 14.418979004549637


In [47]:
proposed_policy = np.where(solution >= 0.5, 1, 0)

In [48]:
proposed_policy

array([1, 0, 0, ..., 0, 0, 1])

In [49]:
discount_all = np.mean(revenue_fn(train_data,  0.0, 0.15, 1, np.zeros(train_data.shape[0])))
no_discount = np.mean(revenue_fn(train_data,  0.0, 0.15, 1, np.ones(train_data.shape[0])))

In [53]:
print("Current policy {:.4f}".format(np.mean(train_data['demand']*train_data['price'])))
print("Discount all {:.4f}".format(discount_all))
print("No discount {:.4f}".format(no_discount))
print("-"*100)
print("Proposed strategy {:.4f}".format(solution_fitness))

Current policy 14.3149
Discount all 13.4095
No discount 14.2536
----------------------------------------------------------------------------------------------------
Proposed strategy 14.4190


In [54]:
from collections import Counter 
print(Counter(proposed_policy))
print(Counter(train_data.treatment.values))


Counter({0: 2671, 1: 2329})
Counter({0: 2841, 1: 2159})


In [55]:
current_policy = train_data.treatment.values

In [56]:
proposed_policy = [str(x) for x in proposed_policy]
current_policy = [str(x) for x in current_policy]

In [57]:
----

SyntaxError: invalid syntax (<ipython-input-57-759c4c5efcf3>, line 1)

In [58]:
best = ga_instance.best_solution()

In [ ]:
a = np.array([best[0]]*1000)
a.shape


In [ ]:
init = np.array([best[0]]*1000)

In [ ]:
function_inputs = np.random.rand(train_data.shape[0]) # Function inputs.

num_generations = 5000 # Number of generations.
num_parents_mating = 500 # Number of solutions to be selected as parents in the mating pool.

# To prepare the initial population, there are 2 ways:
# 1) Prepare it yourself and pass it to the initial_population parameter. This way is useful when the user wants to start the genetic algorithm with a custom initial population.
# 2) Assign valid integer values to the sol_per_pop and num_genes parameters. If the initial_population parameter exists, then the sol_per_pop and num_genes parameters are useless.
sol_per_pop = 1500 # Number of solutions in the population.
num_genes = len(function_inputs)

init_range_low = -1
init_range_high = 1

parent_selection_type = "sss" # Type of parent selection.
keep_parents = -1 # Number of parents to keep in the next population. -1 means keep all parents and 0 means keep nothing.

crossover_type = "two_points" # Type of the crossover operator.

# Parameters of the mutation operation.
mutation_type = "random" # Type of the mutation operator.
mutation_percent_genes = 90 # Percentage of genes to mutate. This parameter has no action if the parameter mutation_num_genes exists or when mutation_type is None.

last_fitness = 0


def callback_generation(ga_instance):
    global last_fitness
    print("Generation = {generation}".format(generation=ga_instance.generations_completed))
    print("Fitness    = {fitness}".format(fitness=ga_instance.best_solution()[1]))
    print("Change     = {change}".format(change=ga_instance.best_solution()[1] - last_fitness))
    print("\n")
    last_fitness = ga_instance.best_solution()[1]

print("Running model")
# Creating an instance of the GA class inside the ga module. Some parameters are initialized within the constructor.
ga_instance = pygad.GA(initial_population = init,
                       num_generations=num_generations,
                       num_parents_mating=num_parents_mating, 
                       fitness_func=fitness_function,
                       sol_per_pop=sol_per_pop, 
                       num_genes=num_genes,
                       init_range_low=init_range_low,
                       init_range_high=init_range_high,
                       parent_selection_type=parent_selection_type,
                       keep_parents=keep_parents,
                       crossover_type=crossover_type,
                       mutation_type=mutation_type,
                       mutation_percent_genes=mutation_percent_genes,
                       callback_generation=callback_generation)

# Running the GA to optimize the parameters of the function.
ga_instance.run()